<p style="font-family: 'Meiryo UI'; font-size: 32px; padding: 25px; text-align: center; color: #ffffff; border-radius: 25px;  font-weight: bold; background-color: #005080;">📚 Hyperparameter Optimization with the MS-EPSO 🐦</p>

## Hyperparameter Optimization Problem

Hyperparameter optimization is the process of finding the optimal set of hyperparameters that result in the best performance of a model. This is an important step in the machine learning pipeline as it can lead to significant improvements in model accuracy and generalization.

Traditional methods for hyperparameter optimization include:

1. **Grid Search**: This method involves exhaustively searching through a manually specified subset of the hyperparameter space. While simple to implement, grid search is computationally expensive and not feasible for large hyperparameter spaces.

2. **Bayesian Optimization**: This method builds a probabilistic model of the objective function and uses it to select the most promising hyperparameters to evaluate in the true objective function. While more efficient, it can be complex to implement and computationally intensive.

In this project, we will test my published Swarm Intelligence algorithm named [Maximum Search Limitations Evolutionary Particle Swarm Optimization (MS-EPSO)](https://link.springer.com/chapter/10.1007/978-3-030-30241-2_59) for hyperparameter tuning.

<p style="font-family: 'Meiryo UI'; font-size: 30px; padding: 12px; text-align: center; color: #ffffff; border-radius: 15px;  font-weight: bold; background-color: #007040;">📚 Libraries</p>

In this experiment, we will use the XGBoost algorithm as our base model. We will utilize the original implementation of the MS-EPSO algorithm, which can be found [here](https://github.com/MtrsN/MS-EPSO).

In [16]:
import json

import numpy as np
import pandas as pd
import xgboost as xgb

from msepso import msepso
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

<p style="font-family: 'Meiryo UI'; font-size: 30px; padding: 12px; text-align: center; color: #ffffff; border-radius: 15px;  font-weight: bold; background-color: #007040;">📊 Dataset</p>

In this notebook, we will use the Breast Cancer dataset. This dataset is simple and comes pre-processed. We will load the data from the sklearn environment.

In [4]:
data = load_breast_cancer()

df = pd.DataFrame(data.data, columns=data.feature_names)

df['Target'] = data.target

df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [5]:
print("The dataset has {} rows and {} columns".format(df.shape[0], df.shape[1]))

The dataset has 569 rows and 31 columns


<p style="font-family: 'Meiryo UI'; font-size: 30px; padding: 12px; text-align: center; color: #ffffff; border-radius: 15px;  font-weight: bold; background-color: #007040;">Why MS-EPSO❓</p>

In recent years, Optuna has established itself as a valuable hyperparameter optimization framework. This framework allows the implementation of various state-of-the-art optimization methods for tuning machine learning models. Among these techniques, the CMA-ES algorithm is notably included.

MS-EPSO was developed to address specific issues found in the Evolutionary Particle Swarm Optimization (EPSO) algorithm. MS-EPSO enhances the balance between exploration (searching through the parameter space) and exploitation (refining the best solutions found). 

MS-EPSO was initially tailor made for a specific application, but it has proven to be a viable solution for general optimization problems, particularly those where the variables exhibit low standard deviation. For instance, it is highly effective in Feature Selection problems, where the decision is binary (i.e., to include or not include a variable). This makes MS-EPSO an interesting choice for tasks.

<p style="font-family: 'Meiryo UI'; font-size: 30px; padding: 12px; text-align: center; color: #ffffff; border-radius: 15px;  font-weight: bold; background-color: #007040;">🚩 The Problem</p>

In order to apply MS-EPSO to any problem, we have to define:

1. Objective function
2. Boundaries
3. Algorithm parameters

However, the hyperparameter problem requires several settings, not limited to:

1. Model
2. Model Parameters
3. Metric
4. Dataset

In [10]:
'''Split the data into training, validation, and test sets. With a larger dataset, we could check for the possibility to stratify the split based on the target variable. 
This would ensure that the distribution of the target variable is similar across all the splits. However, in this case, we will keep it simple and split the data randomly.
'''

RANDOM_SEED = 101

X = df.drop('Target', axis=1)
y = df['Target']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size= 0.2, random_state=RANDOM_SEED)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size= 0.5, random_state=RANDOM_SEED)

print(f'Training set: {X_train.shape}, {y_train.shape}')
print(f'Validation set: {X_val.shape}, {y_val.shape}')
print(f'Test set: {X_test.shape}, {y_test.shape}')

Training set: (455, 30), (455,)
Validation set: (57, 30), (57,)
Test set: (57, 30), (57,)


In [18]:
'''Create our objective function based on MS-EPSO parameters and XGBoost performance.'''

def objective_function(x):

    policy_index = int(x[9])

    params = {

        'verbosity': 0,
        'seed' : RANDOM_SEED,
        'eval_metric': 'logloss',

        'max_depth': int(x[0]),
        'min_child_weight': int(x[1]),
        'n_estimators': int(x[2]),
        'gamma': x[3],
        'subsample': x[4],
        'reg_alpha': x[5],
        'reg_lambda': x[6],
        'learning_rate': x[7],
        'colsample_bytree': x[8],
        'grow_policy':  ['grow_policy', "depthwise", "lossguide"][policy_index]
    }

    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    
    acc = accuracy_score(y_val, y_pred)

    return acc

lower_bounds = [0, 1, 500, 1e-9, 0.3, 1e-9, 1e-9, 0.01, 0.3, 0]
upper_bounds = [12, 7, 1000, 0.5, 1.0, 100.0, 100.0, 1.0, 1.0, 2]

example_solution = [
        0.5, # max_depth
        3, # min_child_weight
        1, # n_estimators
        0.5, # gamma
        0.5, # subsample
        0.5, # reg_alpha
        0.5, # reg_lambda
        0.5, # learning_rate
        0.5, # colsample_bytree
        1 # grow_policy
]


# Test our objective function
example_model, example_solution_accuracy = objective_function(example_solution)

print(f"Example Solution:{example_solution}")
print(f"Example Accuracy:{example_solution_accuracy}")

Example Solution:[0.5, 3, 1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1]
Example Accuracy:0.9649122807017544


In [19]:
    params = {

        'verbosity': 0,
        'seed' : RANDOM_SEED,
        'eval_metric': 'logloss',

        'max_depth': int(example_solution[0]),
        'min_child_weight': int(example_solution[1]),
        'n_estimators': int(example_solution[2]),
        'gamma': example_solution[3],
        'subsample': example_solution[4],
        'reg_alpha': example_solution[5],
        'reg_lambda': v[6],
        'learning_rate': x[7],
        'colsample_bytree': x[8],
        'grow_policy':  ['grow_policy', "depthwise", "lossguide"][policy_index]
    }

odel = XGBClassifier(**example_solution)
model.fit(X_train, y_train)

AttributeError: 'list' object has no attribute 'update'